In [50]:
from __future__ import division
import pandas as pd
import pandas_datareader as pdr
import numpy as np
from numpy.linalg import inv,pinv
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import matplotlib
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
matplotlib.style.use('ggplot')

# Purpose of the Notebook

Factor Risk Parity Prototype

In [2]:
country_etfs = pd.read_csv('../Data/country_data/ETF_adj_close.csv',index_col=0)
country_etfs.index = pd.to_datetime(country_etfs.index)
ff5 = pdr.famafrench.FamaFrenchReader('Emerging_5_Factors',start = '2000-01-01').read()[0]
ff_mom = pdr.famafrench.FamaFrenchReader('Emerging_MOM_Factor',start = '2000-01-01').read()[0]
ff6 = pd.concat([ff5,ff_mom],axis=1)

In [4]:
# find monthly return - take last date and pct-change
country_returns = country_etfs.groupby(pd.Grouper(freq = 'M')).last().pct_change()
country_returns.index = country_returns.index.to_period('M')

# Step 1: Obtain Loading and Covariance Matrix per Cluster

In [44]:
rolling_betas = {}
# ff5_new = ff5.copy()
# ff5_new['Mkt'] = ff5_new['Mkt-RF'] + ff5_new['RF']
#cols = ['SMB', 'HML', 'RMW', 'CMA', 'Mkt']
cols = ['SMB', 'HML', 'WML']
for c in country_returns:
    y = country_returns.loc[ff6.index][c]
    y = y*100-ff6['Mkt-RF'] - ff6['RF']
    y = y.dropna()
    X = ff6.loc[y.index][cols]
    X = sm.add_constant(X, prepend=False)
    mod = RollingOLS(y, X,window=12)
    rolling_res = mod.fit(params_only = True)
    rolling_betas[c] = rolling_res.params

In [45]:
rolling_df = pd.concat(rolling_betas)
rolling_df

SMB       HML       WML     const
                     Date                                           
Argentina            2011-04       NaN       NaN       NaN       NaN
                     2011-05       NaN       NaN       NaN       NaN
                     2011-06       NaN       NaN       NaN       NaN
                     2011-07       NaN       NaN       NaN       NaN
                     2011-08       NaN       NaN       NaN       NaN
...                                ...       ...       ...       ...
United Arab Emirates 2020-08  1.329554 -0.205316 -0.099375 -2.732862
                     2020-09  1.268134  0.042316 -0.075638 -1.947614
                     2020-10  1.316618  0.045517 -0.035861 -2.151875
                     2020-11  1.366387 -0.041240 -0.215107 -1.901273
                     2020-12  1.364707 -0.140098 -0.316254 -1.693281

[3383 rows x 4 columns]

In [46]:
months_list = rolling_df.index.get_level_values(1).unique()
monthly_rtn, monthly_cov = {}, {}

for m in months_list:
    month_df = rolling_df.loc[(slice(None), m), :].iloc[:,:3]
    monthly_rtn[m] = month_df.mean(axis=0)
    monthly_cov[m] = month_df.cov()

In [ ]:
 # risk budgeting optimization
def calculate_portfolio_var(w,V):
    # function that calculates portfolio risk
    w = np.matrix(w)
    return (w*V*w.T)[0,0]

def calculate_risk_contribution(w,V):
    # function that calculates asset contribution to total risk
    w = np.matrix(w)
    sigma = np.sqrt(calculate_portfolio_var(w,V))
    # Marginal Risk Contribution
    MRC = V*w.T
    # Risk Contribution
    RC = np.multiply(MRC,w.T)/sigma
    return RC

def risk_budget_objective(x,pars):
    # calculate portfolio risk
    V = pars[0]# covariance table
    x_t = pars[1] # risk target in percent of portfolio risk
    sig_p =  np.sqrt(calculate_portfolio_var(x,V)) # portfolio sigma
    risk_target = np.asmatrix(np.multiply(sig_p,x_t))
    asset_RC = calculate_risk_contribution(x,V)
    J = sum(np.square(asset_RC-risk_target.T))[0,0] # sum of squared error
    return J

def total_weight_constraint(x):
    return np.sum(x)-1.0

def long_only_constraint(x):
    return x

x_t = [0.25, 0.25, 0.25, 0.25] # your risk budget percent of total portfolio risk (equal risk)
cons = ({'type': 'eq', 'fun': total_weight_constraint},
{'type': 'ineq', 'fun': long_only_constraint})
res= minimize(risk_budget_objective, w0, args=[V,x_t], method='SLSQP',constraints=cons, options={'disp': True})
w_rb = np.asmatrix(res.x)